In [ ]:
import os
import json

def collect_dataset_info(root_dir):
    dataset_info = []

    for mode in ['train', 'val', 'test']:
        mode_dir = os.path.join(root_dir, mode)
        if not os.path.exists(mode_dir):
            continue

        for video_folder in os.listdir(mode_dir):
            video_id = video_folder  # Folder name is used as video_id
            video_folder_path = os.path.join(mode_dir, video_folder)

            if os.path.isdir(video_folder_path):
                video_path, audio_path, motion_path = None, None, None

                for file_name in os.listdir(video_folder_path):
                    file_path = os.path.join(video_folder_path, file_name)

                    if file_name.endswith('.mp4'):
                        video_path = file_path
                    elif file_name.endswith('.wav'):
                        audio_path = file_path
                    elif file_name.endswith('.pkl'):
                        motion_path = file_path

                # Create an entry only if all the necessary files are present
                if video_path and audio_path and motion_path:
                    dataset_info.append({
                        "video_id": video_id,
                        "video_path": video_path,
                        "audio_path": audio_path,
                        "motion_path": motion_path,
                        "mode": mode
                    })

    return dataset_info


# Set the root directory path of your dataset
root_dir = '/path/to/ExpressiveWholeBodyDatasetReleaseV1.0'
dataset_info = collect_dataset_info(root_dir)
output_file = 'dataset_info.json'
with open(output_file, 'w') as json_file:
    json.dump(dataset_info, json_file, indent=4)
print(f"Dataset information saved to {output_file}")


import os
import json
import pickle
import wave

def load_pkl(pkl_path):
    try:
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        return data
    except Exception as e:
        print(f"Error loading {pkl_path}: {e}")
        return None

def load_wav(wav_path):
    try:
        with wave.open(wav_path, 'rb') as f:
            frames = f.getnframes()
        return frames
    except Exception as e:
        print(f"Error loading {wav_path}: {e}")
        return None

def generate_clips(data, stride, window_length):
    clips = []
    for entry in data:
        pkl_data = load_pkl(entry['motion_path'])
        wav_frames = load_wav(entry['audio_path'])

        # Only continue if both the pkl and wav files are successfully loaded
        if pkl_data is None or wav_frames is None:
            continue

        # Determine the total length of the sequence from pkl data
        total_frames = len(pkl_data)  # Assuming pkl contains motion data frames

        # Generate clips based on stride and window_length
        for start_idx in range(0, total_frames - window_length + 1, stride):
            end_idx = start_idx + window_length
            clip = {
                "video_id": entry["video_id"],
                "video_path": entry["video_path"],
                "audio_path": entry["audio_path"],
                "motion_path": entry["motion_path"],
                "mode": entry["mode"],
                "start_idx": start_idx,
                "end_idx": end_idx
            }
            clips.append(clip)

    return clips


# Load the existing dataset JSON file
input_json = 'dataset_info.json'
with open(input_json, 'r') as f:
    dataset_info = json.load(f)

# Set stride and window length
stride = 5  # Adjust stride as needed
window_length = 10  # Adjust window length as needed

# Generate clips for all data
clips_data = generate_clips(dataset_info, stride, window_length)

# Save the filtered clips data to a new JSON file
output_json = 'filtered_clips_data.json'
with open(output_json, 'w') as f:
    json.dump(clips_data, f, indent=4)

print(f"Filtered clips data saved to {output_json}")

In [1]:
import torch
import pickle

with open("/Users/haiyang/download_backup/oliver/Abortion_Laws_-_Last_Week_Tonight_with_John_Oliver_HBO-DRauXXz6t0Y.webm/test/214438-00_07_16-00_07_26/214438-00_07_16-00_07_26.pkl", "rb") as f:
    # Load the file by mapping any GPU tensors to CPU
    pkl_example = torch.load(f, map_location=torch.device('cpu'))

# Now check the type of the object
print(type(pkl_example))

# If it's a dictionary, print its keys
if isinstance(pkl_example, dict):
    print(pkl_example.keys())


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/haiyang/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/haiyang/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/haiyang/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.sta

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.